In [2]:
import cv2
import numpy as np
import os
from zipfile import ZipFile
import shutil

In [45]:
archieve = 'tlights.zip'
unpacked = 'unpacked'

with ZipFile(archieve, 'r') as zip_ref:
    zip_ref.extractall(unpacked)

In [46]:
tlights_folder = os.path.join(unpacked, 'tlights')
file_list = os.listdir(tlights_folder)
file_list.sort()

output_folder = {
    "red": os.path.join(unpacked, 'red_signals'),
    "green": os.path.join(unpacked, 'green_signals'),
    "yellow": os.path.join(unpacked, 'yellow_signals'),
    "unclassifiable": os.path.join(unpacked, 'unclassifiable_signals')
    }

for dir_path in output_folder.values():
    os.makedirs(dir_path, exist_ok=True)

In [47]:
def classify_traffic_light(image_path):
    # Load the image
    img = cv2.imread(image_path)
    

    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    
    h, w = img.shape[0], img.shape[1] 
    center = (w//2, h//2) 
    radius = min(w, h)//2 
    mask = np.zeros((h, w), dtype=np.uint8) 
    cv2.circle(mask, center, radius, (1, 1, 1), thickness=-1)
    

    img = cv2.bitwise_and(img, img, mask=mask)  
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    
    red_lower1 = np.array([0, 50, 50])
    red_upper1 = np.array([10, 255, 255])
    red_lower2 = np.array([160, 50, 50])
    red_upper2 = np.array([180, 255, 255])
    
    yellow_lower = np.array([15, 100, 100])
    yellow_upper = np.array([30, 255, 255])
    
    green_lower = np.array([40, 50, 50])
    green_upper = np.array([90, 255, 255])
    
    
    mask_red1 = cv2.inRange(hsv, red_lower1, red_upper1)
    mask_red2 = cv2.inRange(hsv, red_lower2, red_upper2)
    mask_red = cv2.add(mask_red1, mask_red2)
    mask_yellow = cv2.inRange(hsv, yellow_lower, yellow_upper)
    mask_green = cv2.inRange(hsv, green_lower, green_upper)
    
    
    count_red = cv2.countNonZero(mask_red)
    count_yellow = cv2.countNonZero(mask_yellow)
    count_green = cv2.countNonZero(mask_green)


    max_count = max(count_red, count_yellow, count_green)
    if max_count < 50:  
        return "unclassifiable"
    elif max_count == count_red:
        return "red"
    elif max_count == count_yellow:
        return "yellow"
    elif max_count == count_green:
        return "green"

In [48]:
red_signals = []
green_signals = []
yellow_signals = []
unclassifiable_signals = []

In [49]:
for file_name in file_list:
    file_path = os.path.join(tlights_folder, file_name)
    classification = classify_traffic_light(file_path)
    
    output_dir = output_folder[classification]
    shutil.move(file_path, os.path.join(output_dir, file_name))

    if classification == "red":
        red_signals.append(file_name)
    elif classification == "green":
        green_signals.append(file_name)
    elif classification == "yellow":
        yellow_signals.append(file_name)
    else:
        unclassifiable_signals.append(file_name)

In [50]:
print("Red signals:", len(red_signals))
print("Green signals:", len(green_signals))
print("Yellow signals:", len(yellow_signals))
print("Unclassifiable signals:", len(unclassifiable_signals))

Red signals: 14
Green signals: 32
Yellow signals: 3
Unclassifiable signals: 1
